1. import data from ACLED
2. Connect to google sheets
3. paste to google sheets
4.

# Google Sheet Credentials

In [4]:
import pandas as pd
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [5]:
r = requests.get("http://acleddata.com/api/acled/read")

In [50]:
data = r.json()

In [51]:
data = data['data']

In [24]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('ACLED_to_Google_Sheets-e244e52c2f43.json', scope)

In [71]:
gc = gspread.authorize(credentials)

In [72]:
sheet = gc.open('acled_test_4').sheet1

#### 1. Create a dataframe from data
2. Send columns to sheet
3. Send data to sheet

In [56]:
#create dataframe

df = pd.DataFrame(data)

In [65]:
#function to count range for cells
def numberToLetters(q):
    q = q -1
    result = ''
    while q>= 0:
        remain = q % 26
        result = chr(remain+65) + result;
        q = q//26 - 1
    return result

In [81]:
#Adds column headers
columns = df.columns.values.tolist()
cell_list = sheet.range('A1:'+numberToLetters(len(columns))+'1')
for cell in cell_list:
    val = columns[cell.col-1]
    if type(val) is str:
        val = val.decode('utf-8')
    cell.value = val
sheet.update_cells(cell_list)

In [82]:
cell_list = sheet.range('A2:'+numberToLetters(num_columns)+str(num_lines+1))
for cell in cell_list:
    val = df.iloc[cell.row-2, cell.col-1]
    if type(val) is str:
        val = val.decode('utf-8')
    elif isinstance(val,(int, long, float, complex)):
        val= int(round(val))
    cell.value = val
sheet.update_cells(cell_list)

## the main sheet upload

In [10]:
df_acled_complete = pd.read_excel('acledmaster_tosql.xlsx', sheetname=0)

In [33]:
cell = cell_list[0]
cell

<Cell R503C1 ''>

In [63]:
df_partial = df_acled_complete[:999]

In [64]:
num_lines, num_columns = df_partial.shape
cell_list = sheet.range('A2:'+numberToLetters(num_columns)+str(num_lines+1))
for cell in cell_list:
    val = df_partial.iloc[cell.row-2, cell.col-1]
    if type(val) is str:
        val = val.decode('utf-8')
    #elif isinstance(val,(int, long, float, complex)):
       # val= int(round(val))
    cell.value = val
sheet.update_cells(cell_list)

# What I need to do
1. Write iterator that changes the chunks of spreadsheet to uploaded to google sheets
2. Connect to script that posts to API
3. Add headers
4. compare headers from acled api to dataframe headers

In [76]:
start = 0
end = 500
start_col = 'A'
start_row = 2
while start < 67000:
    df_input = df_acled_complete[start:end]
    num_lines, num_columns = df_input.shape
    start_col_row = start_col + str(start_row)+":"
    cell_list = sheet.range(start_col_row+numberToLetters(num_columns)+str(num_lines+1))
    for cell in cell_list:
        val = df_input.iloc[cell.row-2, cell.col-1]
        if type(val) is str:
            val = val.decode('utf-8')
    #elif isinstance(val,(int, long, float, complex)):
       # val= int(round(val))
        cell.value = val
    sheet.update_cells(cell_list)
    start += 500
    end += 500
    start_row +=500
    
    

IndexError: single positional indexer is out-of-bounds

In [68]:
start_col = 'A'
start_row = 2

In [70]:
start_col + str(start_row)+":"

'A2:'